In [2]:
################################################################################
## Code adapted from demo_caiman_cnmf_3D as imported from github 21/11/2018
## https://github.com/flatironinstitute/CaImAn
################################################################################

import cde_cell_functions as cc

# Import relevant packages
#===============================================================================
import imp
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil
import shutil
from scipy.ndimage.filters import gaussian_filter
import scipy.sparse
import sys
import re
from skimage.external.tifffile import imread
from skimage import io
import warnings 
from IPython.display import clear_output
import copy
from importlib import reload
from PIL import Image
import datetime

# Caiman setup
#-------------------------------------------------------------------------------
import caiman as cm
import caiman.source_extraction.cnmf as cnmf
from caiman.utils.visualization import nb_view_patches3d
from caiman.source_extraction.cnmf import params as params
from caiman.components_evaluation import evaluate_components, estimate_components_quality_auto
from caiman.motion_correction import MotionCorrect
from caiman.cluster import setup_cluster
from caiman.paths import caiman_datadir
from caiman.utils.visualization import inspect_correlation_pnr

# Jupyter specific autoreloading for external functions (in case changes are made)
# %load_ext autoreload
# %autoreload

In [3]:
# Housekeeping
#===============================================================================
# Module flags

display_movie   = False      # play movie of tifs that are loaded
save_results    = False      # flag to save results or not

# Define folder locations
#-------------------------------------------------------------------------------
reload(cc)
Fbase     = '/Users/roschkoenig/Dropbox/Research/1812 Critical Dynamics Epilepsy'
Fsave     = '/Users/roschkoenig/Dropbox/Research/1812 Critical Dynamics Epilepsy Data'
Fscripts  = Fbase + os.sep + '03 - Cell detection' 
Fdata     = '/Volumes/ALBERS/1812 Critical Dynamics in Epilepsy'
Zfish     = cc.cde_cell_fishspec(Fdata, 'RM')

In [12]:
Fish = Zfish[-2:]
Fish[0]["Name"]

'ZFRR014_02'

# Split images up into planes

In [13]:
for z in Fish:
    print('----------------------------------------------------------------------------')
    print('Currently working on ' + z["Name"])
    print('----------------------------------------------------------------------------')
    cc.cde_cell_planesave(Fdata, z, mxpf = 7000)

----------------------------------------------------------------------------
Currently working on ZFRR014_02
----------------------------------------------------------------------------
I found 2 Conditions
Condition BL
> There are 9 Planes
> Processing plane 1
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-06-12 16:30:42| Saving interim file
>  2019-06-12 16:31:36| Saving interim file
> Processing plane 2
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-06-12 18:22:33| Saving interim file
>  2019-06-12 18:23:23| Saving interim file
> Processing plane 3
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-06-12 20:15:46| Saving interim file
>  2019-06-12 20:16:36| Saving interim file
> Processing plane 4
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-06-12 22:08:48| Saving interim file
>  2019-06-12 22:09:40| Saving interim file
> Processing plane 5
>  >  >  >  >  >  >  >  >  >  >  >  >  >  2019-06-13 00:03:46| Saving interim file
>  2019-06-13 00:04:39| Saving interim file
> Processing plane 

# Run actual cell segmentation

In [4]:
imp.reload(cc),  
Pfish = cc.cde_cell_fishspec(Fdata, 'PL')
f     = 3

In [20]:
for f in range(len(Pfish)):
    for c in range(len(Pfish[f]["Cond"])):
        planes = Pfish[f]["Cond"][c]["Plane"]
        Estimates = []
        for p in range(len(planes)-1): # That last plane is a bitch
            fname = planes[p]["Tpaths"]
            fr = 4                                                              # frame rate (Hz)
            decay_time = 0.5                                                    # approximate length of transient event in seconds
            gSig = (4,4)                                                        # expected half size of neurons
            p = 1                                                               # order of AR indicator dynamics
            min_SNR = 1                                                         # minimum SNR for accepting new components
            rval_thr = 0.90                                                     # correlation threshold for new component inclusion
            ds_factor = 1                                                       # spatial downsampling factor (increases speed but may lose some fine structure)
            gnb = 2                                                             # number of background components
            gSig = tuple(np.ceil(np.array(gSig)/ds_factor).astype('int'))       # recompute gSig if downsampling is involved
            mot_corr = True                                                     # flag for online motion correction 
            pw_rigid = False                                                    # flag for pw-rigid motion correction (slower but potentially more accurate)
            max_shifts_online = np.ceil(10./ds_factor).astype('int')            # maximum allowed shift during motion correction
            sniper_mode = True                                                  # flag using a CNN to detect new neurons (o/w space correlation is used)
            init_batch = 200                                                    # number of frames for initialization (presumably from the first file)
            expected_comps = 500                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
            dist_shape_update = True                                            # flag for updating shapes in a distributed way
            min_num_trial = 10                                                  # number of candidate components per frame     
            K = 2                                                               # initial number of components
            epochs = 2                                                          # number of passes over the data
            show_movie = False                                                  # show the movie with the results as the data gets processed

            params_dict = {'fnames': fname,
                           'fr': fr,
                           'decay_time': decay_time,
                           'gSig': gSig,
                           'p': p,
                           'min_SNR': min_SNR,
                           'rval_thr': rval_thr,
                           'ds_factor': ds_factor,
                           'nb': gnb,
                           'motion_correct': mot_corr,
                           'init_batch': init_batch,
                           'init_method': 'bare',
                           'normalize': True,
                           'expected_comps': expected_comps,
                           'sniper_mode': sniper_mode,
                           'dist_shape_update' : dist_shape_update,
                           'min_num_trial': min_num_trial,
                           'K': K,
                           'epochs': epochs,
                           'max_shifts_online': max_shifts_online,
                           'pw_rigid': pw_rigid,
                           'show_movie': show_movie}
            opts = cnmf.params.CNMFParams(params_dict=params_dict)
            clear_output()
            print('-----------------------------------------------------------------------')
            print('Currently processing condition ' + Pfish[f]["Cond"][c]["Name"])
            print('> Plane ' + str(p) + ' of ' + str(len(planes)))
            print('-----------------------------------------------------------------------')
            cmn  = cnmf.online_cnmf.OnACID(params=opts)
            cmn.fit_online()
            Estimates.append({'Spatial':cmn.estimates.A,'Temporal':cmn.estimates.C,'Background':cmn.estimates.b})
            
        Pfish[f]["Cond"][c].update({"CMN":Estimates})
        
    # Save everyhting into folder
    #---------------------------------------------------------------------------------
    Fcmn = Fsave + os.sep + 'Analysis' + os.sep + 'CMN' + os.sep + Pfish[f]["Name"]
    if not os.path.exists(Fcmn): os.makedirs(Fcmn)

    for c in range(len(Pfish[f]["Cond"])):
        Fccond = Fcmn + os.sep + Pfish[f]["Cond"][c]["Name"]
        if not os.path.exists(Fccond):
            os.makedirs(Fccond)
        for p in range(len(Pfish[f]["Cond"][c]["CMN"])):
            scipy.io.savemat(Fccond + os.sep + Pfish[f]["Name"] + '_P' + str(p).zfill(2), Pfish[f]["Cond"][c]["CMN"][p])
            
            

-----------------------------------------------------------------------
Currently processing condition BL
> Plane 1 of 9
-----------------------------------------------------------------------
Size frame:(298, 451)
Noise Normalization
Roi Extraction...
Greedy initialization of spatial and temporal components using spatial Gaussian filtering
USING TOTAL SUM FOR INITIALIZATION....
(Hals) Refining Components...
Expecting 500 components
Now processing file /Volumes/ALBERS/1812 Critical Dynamics in Epilepsy/PL_ZFRR003_01/BL/ZFRR003_01_s01_B_PL00.tif
Epoch: 1. 200 frames have beeen processed in total. 0 new components were added. Total # of components is 2


error: OpenCV(4.1.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/templmatch.cpp:1112: error: (-215:Assertion failed) _img.size().height <= _templ.size().height && _img.size().width <= _templ.size().width in function 'matchTemplate'


In [301]:
# Save everyhting into folder
#---------------------------------------------------------------------------------
Fcmn = Fsave + os.sep + 'Analysis' + os.sep + 'CMN' + os.sep + Pfish[f]["Name"]
if not os.path.exists(Fcmn): os.makedirs(Fcmn)
    
for c in range(len(Pfish[f]["Cond"])):
    Fccond = Fcmn + os.sep + Pfish[f]["Cond"][c]["Name"]
    if not os.path.exists(Fccond):
        os.makedirs(Fccond)
    for p in range(len(Pfish[f]["Cond"][c]["CMN"])):
        scipy.io.savemat(Fccond + os.sep + Pfish[f]["Name"] + '_P' + str(p).zfill(2), Pfish[f]["Cond"][c]["CMN"][p])

In [253]:
Estimates = Secure_Estimates[0:9]
Pfish[f]["Cond"][0].update({"CMN":Estimates})

In [19]:
Pfish[0]['Cond'][0]['Path']

'/Volumes/ALBERS/1812 Critical Dynamics in Epilepsy/PL_ZFRR010_01/BL'